<a href="https://colab.research.google.com/github/KilopSahani/RahulKumarSahani-/blob/main/LLM_WebToGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -r /content/LLM-WebToGraph/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 23.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 21.9 MB/s eta 0:00:00
     ━

In [2]:
pip install uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [3]:
!pip install python-dotenv

In [4]:
pip install fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.7 MB/s eta 0:00:00


In [19]:
!pip install tenacity

In [21]:
import os

import backoff
import openai  # for OpenAI API calls
from dotenv import load_dotenv
from langchain.chains import create_extraction_chain
from langchain.chat_models import ChatOpenAI
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

from m1.src.app import utils
from m1.src.components.base_component import BaseComponent

load_dotenv()


def get_schema():
    """
    The get_schema function reads the schema.yml file and returns a dictionary of the schema.

    :return: The schema
    :doc-author: Trelent
    """
    schema = utils.read_yaml_file('services/schema.yml')
    return schema


class Llm(BaseComponent):

    def __init__(self, model: str):
        super().__init__('Lllm')
        self.model = model
        # for huggingface hub models
        # self.llm = HuggingFaceHub(repo_id='ValiantLabs/ShiningValiant', task='text-generation',
        #                           huggingfacehub_api_token=os.getenv('HF_AUTH_TOKEN'),
        #                           model_kwargs={"temperature": 0, "max_length": 64})
        self.llm = ChatOpenAI(temperature=0, model_name=model, openai_api_key=os.getenv('OPENAI_API_KEY'))

    @backoff.on_exception(backoff.expo, Exception)
    @retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
    def run(self, input_text):
        """
        The run function is the main entry point for your component.
        It will be called with a string of text to process, and should return a dictionary of results.
        The keys in this dictionary are the names of slots that you defined in your schema.

        :param self: Represent the instance of the class
        :param input_text: Pass the text that we want to extract entities from
        :return: A dictionary with the following structure:
        """
        schema = get_schema()
        self.logger.info(f'schema: {schema}')
        chain = create_extraction_chain(schema, self.llm)
        llm_response = chain.run(input_text)
        self.logger.info(f'llm_response: {llm_response}')
        return llm_response

In [34]:
from typing import Union, List

from langchain.document_loaders import AsyncHtmlLoader
from langchain.document_transformers import BeautifulSoupTransformer
from m1.src.app import utils
from m1.src.app.llm import Llm
from m1.src.components.base_component import BaseComponent
from m1.src.datalayer.Neo4jDumper import Neo4jDumper


class NameIdentityRetrievalForHtml(BaseComponent):
    def __init__(self, model_name, data_path):

        """
        The __init__ function is called when the class is instantiated.
        It sets up the initial values of all attributes for an instance of a class.
        The self parameter refers to the current instance of a class, and it's required by Python.

        :param self: Represent the instance of the class
        :param model_name: Specify the model name that we want to use for our predictions
        :param data_path: Read the yaml file which contains the links to be scraped
        :return: Nothing
        """
        super().__init__('NameIdentityRetrievalForHtml')
        self.sources = utils.read_yaml_file(data_path)
        self.html_sources = self.sources.get('link', [])
        # instantiating the openai llm model and neo4j connection
        self.neo4j_instance = Neo4jDumper(config_path='app/config.yml')
        self.open_ai_llm = Llm(model=model_name)

    def run_async(self, **kwargs):

        """
        The run_async function is used to run the pipeline asynchronously.
            It takes in a list of html sources and extracts knowledge graph from them using openai api.
            The extracted knowledge graph is then dumped into neo4j database.

        :param self: Represent the instance of the object itself
        :param **kwargs: Pass a variable number of keyword arguments to a function
        :return: A list of all the knowledge graphs extracted from the html sources
        """
        for link in self.html_sources:
            loader =AsyncHtmlLoader(link)
            html = loader.load()
            # html = loader.load()
            bs_transformer = BeautifulSoupTransformer()
            docs_transformed = bs_transformer.transform_documents(html, tags_to_extract=["table"])
            self.logger.info(docs_transformed[0].page_content[0:500])

            # setting up openai model and extracting knowledge graph
            self.logger.info(f'loading model {self.open_ai_llm}')

            # just sending last few lines of csv as the token limit is limited of openai api free version.
            # model should  be changed to claude2 (Anthropic) or premium openai api key should be used.
            # response = self.open_ai_llm.extract_and_store_graph(document=docs_transformed[0])
            tokens_cap = len(docs_transformed[0].page_content) - 4
            response = self.open_ai_llm.run(input_text=docs_transformed[0].page_content[tokens_cap:])
            # instantiating neo4jBD and dumping the knowledge graph
            self.neo4j_instance.run(data=response)
            self.logger.info(f'knowledge graph populated successfully for data source: {link}')

    def run(self, input: Union[str, List[float]]) -> str:
        pass


In [35]:
from typing import List, Optional

from langchain.graphs.graph_document import (
    Node as BaseNode,
    Relationship as BaseRelationship
)
from langchain.pydantic_v1 import Field, BaseModel


class Property(BaseModel):
    """A single property consisting of key and value"""
    key: str = Field(..., description="key")
    value: str = Field(..., description="value")


class Node(BaseNode):
    properties: Optional[List[Property]] = Field(
        None, description="List of node properties")


class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(
        None, description="List of relationship properties"
    )


class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""
    nodes: List[Node] = Field(
        ..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(
        ..., description="List of relationships in the knowledge graph"
    )


def format_property_key(s: str) -> str:
    words = s.split()
    if not words:
        return s
    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]
    return "".join([first_word] + capitalized_words)


def props_to_dict(props) -> dict:
    """Convert properties to a dictionary."""
    properties = {}
    if not props:
        return properties
    for p in props:
        properties[format_property_key(p.key)] = p.value
    return properties


def map_to_base_node(node: Node) -> BaseNode:
    """Map the KnowledgeGraph Node to the base Node."""
    properties = props_to_dict(node.properties) if node.properties else {}
    # Add name property for better Cypher statement generation
    properties["name"] = node.id.title()
    return BaseNode(
        id=node.id.title(), type=node.type.capitalize(), properties=properties
    )


def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Map the KnowledgeGraph Relationship to the base Relationship."""
    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}
    return BaseRelationship(
        source=source, target=target, type=rel.type, properties=properties
    )


/usr/lib/python3.10/tokenize.py:527: RuntimeWarning: coroutine 'Server.serve' was never awaited
  pseudomatch = _compile(PseudoToken).match(line, pos)
ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Task-2' coro=<LifespanOn.main() running at /usr/local/lib/python3.10/dist-packages/uvicorn/lifespan/on.py:86> wait_for=<Future pending cb=[Task.__wakeup()]>>
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/starlette/routing.py", line 741, in lifespan
    await receive()
GeneratorExit

ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Task-4' coro=<LifespanOn.main() running at /usr/local/lib/python3.10/dist-packages/uvicorn/lifespan/on.py:86> wait_for=<Future pending cb=[Task.__wakeup()]>>
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/starlette/routing.py", line 741, in lifespan
    await receive()
GeneratorExit



In [36]:
from langchain.graphs import Neo4jGraph
from langchain.graphs.graph_document import GraphDocument
from neo4j import GraphDatabase

from m1.src.app import utils
from m1.src.components.base_component import BaseComponent
from m1.src.datalayer.KnowledgeGraph import map_to_base_node, map_to_base_relationship


class Neo4jDumper(BaseComponent):
    def __init__(self, config_path):
        super().__init__('Neo4jDumper')
        config = utils.read_yaml_file(config_path)
        self.uri = config.get('neo4j').get('uri')
        self.username = config.get('neo4j').get('username')
        self.password = config.get('neo4j').get('password')
        self.graph = Neo4jGraph(
            url=self.uri, username=self.username, password=self.password
        )

    def dump_data(self, tx, data):
        for key, value in data.items():
            # Create a node for each key-value pair
            tx.run(query="CREATE (n:Node {key: $key, value: $value})", key=key, value=value)
            self.logger.info(f"Dumped data for {key}: {value} to neo4j")

    def run(self, data):
        try:
            with GraphDatabase.driver(self.uri, auth=(self.username, self.password)) as driver:
                with driver.session() as session:
                    self.dump_data(session, data)
            self.logger.info("Neo4j database connected successfully. and data dumped successfully.")
        except Exception as e:
            self.logger.error(f"Error while connecting to neo4j: {str(e)}")
        finally:
            session.close()

    # New implementation using graph document
    def run2(self, data, document):
        try:
            graph = Neo4jGraph(
                url=self.uri, username=self.username, password=self.password
            )
            # Construct a graph document
            graph_document = GraphDocument(
                nodes=[map_to_base_node(node) for node in data.nodes],
                relationships=[map_to_base_relationship(rel) for rel in data.rels],
                source=document
            )
            # Store information into a graph
            graph.add_graph_documents([graph_document])

        except Exception as e:
            self.logger.error(f"Error while connecting to neo4j: {str(e)}")


In [38]:
from m1.src.app import utils
from m1.src.components.base_component import BaseComponent

pre_defined_sources = ('link', 'csv')


class DataPreprocessor(BaseComponent):
    def __init__(self, datasource):
        super().__init__('DataPreprocessor')
        self.sources = utils.read_yaml_file(datasource)
        self.csv_sources = self.sources.get('csv', [])
        self.html_sources = self.sources.get('link', [])

    def get_csv_sources(self):
        return self.csv_sources

    def get_html_sources(self):
        return self.html_sources

    def run(self, **kwargs):
        pass


In [41]:
from langchain.chains import GraphCypherQAChain
from m1.src.app.llm import Llm
from m1.src.components.base_component import BaseComponent
from m1.src.datalayer.Neo4jDumper import Neo4jDumper


class CypherQa(BaseComponent):
    def __init__(self, model_name):
        super().__init__('cypher_qa')
        # instantiating the openai llm model and neo4j connection
        self.neo4j_instance = Neo4jDumper(config_path='app/config.yml')
        self.open_ai_llm = Llm(model=model_name)
        # schema = utils.read_yaml_file('services/schemaN.yml')
        # graph_schema = construct_schema(schema,[],[])
        print(self.neo4j_instance.graph.schema)
        self.cypher_chain = GraphCypherQAChain.from_llm(
            cypher_llm=self.open_ai_llm.llm,
            qa_llm=self.open_ai_llm.llm,
            graph=self.neo4j_instance.graph,
            # validate_cypher=True,  # Validate relationship directions
            verbose=True,
        )

    def run(self, text):
        return self.cypher_chain.run(text)


In [27]:
import nest_asyncio
nest_asyncio.apply()

In [43]:
import yaml


def read_yaml_file(file_path):
    """
    Load a YAML file and return its contents as a Python dictionary.

    Args:
        file_path (str): The path to the YAML file.

    Returns:
        dict: A dictionary containing the YAML configuration.
    """
    try:
        with open(file_path, "r") as yaml_file:
            config = yaml.safe_load(yaml_file)
        return config
    except FileNotFoundError:
        print(f"Error: Config file not found at {file_path}")
        return {}
    except yaml.YAMLError as e:
        print(f"Error: Failed to load YAML from {file_path}. {e}")
        return {}





In [76]:
import requests
import streamlit as st

st.title("LLM-WebToGraph")
st.text(
    'This project using langchain and OpenAI LLM to transform data from different sources (weblinks/csv) to knowledge graph and store then in neo4j DB.')
st.write('Press submit to upload process the data and generate knowledge graph.')

if st.button("process csv files and generate knowledge graph"):
    # Send user_input to FastAPI
    fastapi_url = "http://localhost:8000/generate_tags_from_csv"
    response = requests.get(fastapi_url)
    if response.status_code == 200:
        st.write(f"{response.text}")
    else:
        st.error(f"Error: {response.status_code}")

if st.button("process html links and generate knowledge graph"):
    # Send user_input to FastAPI
    fastapi_url = "http://localhost:8000/generate_tags_from_html"
    response = requests.get(fastapi_url)
    if response.status_code == 200:
        st.write(f"{response.text}")
    else:
        st.error(f"Error: {response.status_code}")

user_input = st.text_input("ask any question about data")
if st.button('submit'):
    # Send user_input to FastAPI
    fastapi_url = f"http://localhost:8000/query_graph/{user_input}"
    response = requests.get(fastapi_url)
    if response.status_code == 200:
        st.write(f"{response.text}")
    else:
        st.error(f"Error: {response.status_code}")


In [ ]:
import uvicorn
from dotenv import load_dotenv
from fastapi import FastAPI
from fastapi.responses import HTMLResponse

from m1.src.app import utils
from m1.src.services.Identity_retrival_for_csv import NameIdentityRetrievalForCsv
from m1.src.services.Identity_retrival_for_html import NameIdentityRetrievalForHtml
from m1.src.services.cypher_qa import CypherQa

app = FastAPI(
    title="LLM-WebToGrap",
    description="""This project using langchain and OpenAI LLM to transform data from different sources (web
    links/csv) to knowledge graph and store then in neo4j DB.""",
    version="0.1.0",
)


@app.get("/query_graph/{question}")
def query_graph(question: str):

    """
    The query_graph function takes a question as input and returns the answer to that question.
    The function uses the CypherQa class from the cypher_qachain package, which is an implementation of
    the QAChain algorithm for answering questions about graphs using GPT-3. The model used by this function
    is gpt-3.5-turbo, which was trained on a dataset of ~100k questions and answers about graphs.

    :param question: str: Pass the question to the function
    :return: A htmlresponse object

    """
    graph_cypher_qachain = CypherQa(model_name='gpt-3.5-turbo')
    response = graph_cypher_qachain.run(question)
    return HTMLResponse(content=response, status_code=200)


@app.get("/generate_tags_from_html")
async def generate_tags():
    """
    The generate_tags function is a ReST endpoint that will generate the tags for all the data sources.
    This function is called by an external service, such as Jenkins or Travis CI, to ensure that the tags are up-to-date.
    The function returns a 200 status code if successful and 500 otherwise.

    :return: A htmlresponse object with the content as 'successfully generated the knowledge from the data sources!!!' and status_code as 200

    """
    ner = NameIdentityRetrievalForHtml(model_name='gpt-3.5-turbo', data_path='datalayer/datasources.yml')
    ner.run_async()  # asyncronous call since html pages can take time to load and scrape
    return HTMLResponse(content='Successfully generated the knowledge from the data sources!!!', status_code=200)


@app.get("/generate_tags_from_csv")
def generate_tags():
    """
    The generate_tags function is a ReST endpoint that will generate the tags for each of the data sources.
        It uses the NameIdentityRetrievalForCsv class to accomplish this task.
        The model_name and data_path are passed as parameters to this function.

    :return: A htmlresponse object with the status code 200
    """
    ner = NameIdentityRetrievalForCsv(model_name='gpt-3.5-turbo', data_path='datalayer/datasources.yml')
    ner.run()
    return HTMLResponse(content='Successfully generated the knowledge from the data sources!!!', status_code=200)


# health check route
@app.get("/health")
def health_check():
    return {"status": "healthy"}


if __name__ == '__main__':
    app_config = utils.read_yaml_file('/content/m1/src/app/config.yml')
    load_dotenv()
    uvicorn.run(app, port=app_config.get('port'), host=app_config.get('host'))


INFO:     Started server process [846]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [53]:
!sudo lsof -i -P -n | grep LISTEN

node        7 root   21u  IPv6  19703      0t0  TCP *:8080 (LISTEN)
kernel_ma  22 root    3u  IPv4  18488      0t0  TCP 172.28.0.12:6000 (LISTEN)
colab-fil  60 root    3u  IPv4  18735      0t0  TCP 127.0.0.1:3453 (LISTEN)
jupyter-n 110 root    7u  IPv4  18882      0t0  TCP 172.28.0.12:9000 (LISTEN)
python3   846 root   21u  IPv4  39352      0t0  TCP 127.0.0.1:37689 (LISTEN)
python3   881 root    3u  IPv4  39821      0t0  TCP 127.0.0.1:46561 (LISTEN)
python3   881 root    4u  IPv4  39822      0t0  TCP 127.0.0.1:38643 (LISTEN)


In [46]:
!printenv

SHELL=/bin/bash
NV_LIBCUBLAS_VERSION=12.2.5.6-1
NVIDIA_VISIBLE_DEVICES=all
COLAB_JUPYTER_TRANSPORT=ipc
NV_NVML_DEV_VERSION=12.2.140-1
NV_CUDNN_PACKAGE_NAME=libcudnn8
CGROUP_MEMORY_EVENTS=/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events
NV_LIBNCCL_DEV_PACKAGE=libnccl-dev=2.19.3-1+cuda12.2
NV_LIBNCCL_DEV_PACKAGE_VERSION=2.19.3-1
VM_GCE_METADATA_HOST=169.254.169.253
HOSTNAME=7ef427e43362
LANGUAGE=en_US
TBE_RUNTIME_ADDR=172.28.0.1:8011
COLAB_TPU_1VM=
GCE_METADATA_TIMEOUT=3
NVIDIA_REQUIRE_CUDA=cuda>=12.2 brand=tesla,driver>=470,driver<471 brand=unknown,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=geforce,driver>=470,driver<471 brand=geforcertx,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=titan,driver>=470,driver<471 brand=titanrtx,driver>=470,driver<471 brand=tesla,driver>=525,driver<526 brand=unknown,driver>=525,driver<526 brand=nvidia,driver>=5

In [24]:
import uvicorn
from dotenv import load_dotenv
from fastapi import FastAPI
from fastapi.responses import HTMLResponse

from m1.src.app import utils
from m1.src.services.Identity_retrival_for_csv import NameIdentityRetrievalForCsv
from m1.src.services.Identity_retrival_for_html import NameIdentityRetrievalForHtml
from m1.src.services.cypher_qa import CypherQa

app = FastAPI(
    title="LLM-WebToGrap",
    description="""This project using langchain and OpenAI LLM to transform data from different sources (web
    links/csv) to knowledge graph and store then in neo4j DB.""",
    version="0.1.0",
)


@app.get("/query_graph/{question}")
def query_graph(question: str):

    """
    The query_graph function takes a question as input and returns the answer to that question.
    The function uses the CypherQa class from the cypher_qachain package, which is an implementation of
    the QAChain algorithm for answering questions about graphs using GPT-3. The model used by this function
    is gpt-3.5-turbo, which was trained on a dataset of ~100k questions and answers about graphs.

    :param question: str: Pass the question to the function
    :return: A htmlresponse object

    """
    graph_cypher_qachain = CypherQa(model_name='gpt-3.5-turbo')
    response = graph_cypher_qachain.run(question)
    return HTMLResponse(content=response, status_code=200)


@app.get("/generate_tags_from_html")
async def generate_tags():
    """
    The generate_tags function is a ReST endpoint that will generate the tags for all the data sources.
    This function is called by an external service, such as Jenkins or Travis CI, to ensure that the tags are up-to-date.
    The function returns a 200 status code if successful and 500 otherwise.

    :return: A htmlresponse object with the content as 'successfully generated the knowledge from the data sources!!!' and status_code as 200

    """
    ner = NameIdentityRetrievalForHtml(model_name='gpt-3.5-turbo', data_path='datalayer/datasources.yml')
    ner.run_async()  # asyncronous call since html pages can take time to load and scrape
    return HTMLResponse(content='Successfully generated the knowledge from the data sources!!!', status_code=200)


@app.get("/generate_tags_from_csv")
def generate_tags():
    """
    The generate_tags function is a ReST endpoint that will generate the tags for each of the data sources.
        It uses the NameIdentityRetrievalForCsv class to accomplish this task.
        The model_name and data_path are passed as parameters to this function.

    :return: A htmlresponse object with the status code 200
    """
    ner = NameIdentityRetrievalForCsv(model_name='gpt-3.5-turbo', data_path='datalayer/datasources.yml')
    ner.run()
    return HTMLResponse(content='Successfully generated the knowledge from the data sources!!!', status_code=200)


# health check route
@app.get("/health")
def health_check():
    return {"status": "healthy"}


if __name__ == '__main__':
    app_config = utils.read_yaml_file('app/config.yml')
    load_dotenv()
    uvicorn.run(app, port=app_config.get('port'), host=app_config.get('host'))


Error: Config file not found at app/config.yml


RuntimeError: asyncio.run() cannot be called from a running event loop

In [18]:
import openai

In [19]:
@backoff.on_exception(backoff.expo, openai.error.OpenAIError)

SyntaxError: incomplete input (<ipython-input-19-fd2a75076f53>, line 1)

In [8]:
!pip install backoff retry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 1.7 MB/s eta 0:00:00


In [9]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.2 MB/s eta 0:00:00


In [11]:
!pip install backoff

In [10]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
